In [2]:
import numpy as np
import pandas as pd
import os

data_dir = '../data/'
data_file = 'seq_dg_identity_230404.csv'
data_file = os.path.join(data_dir, data_file)
df = pd.read_csv(data_file, sep='\t')
df.head(3)

,dataset_id,UniProt,protein_sequence,nucleotide_sequence,protein_mutation,nucleic_acid_type,Kd,dG,pH,Tm,...,max_min,max_min_judge,dG_ori,pdb_id,len_p1,len_p2,protein_chain,na_chain,identity_protein_seq,identity_na_seq
0,pnat_12637,P18146,MAAAKAEMQLMSPLQISDPFGSFPHSPTMDNYPKLEEMMLLSNGAP...,GTGTGGGCG|CGCCCACAC,wild,dsDNA,NONE,-0.11,7.5,293.0,...,0.0,good,-0.11,4r2a,A:86,"B:10,C:10",A_1,B_1.C_1,0.14,0.32
1,pnat_12656,P18146,MAAAKAEMQLMSPLQISDPFGSFPHSPTMDNYPKLEEMMLLSNGAP...,GTGTGGGCT|AGCCCACAC,wild,dsDNA,NONE,-0.06,7.5,293.0,...,0.0,good,-0.06,4r2a,A:86,"B:10,C:10",A_1,B_1.C_1,0.14,0.37
2,pnat_1,P16525,MARYDLVDRLNTTFRQMEQELAIFAAHLEQHKLLVARVFSLPEVKK...,TATGTTGTAACTAAAG|CTTTAGTTACAACATA,wild,dsDNA,0.00704,-3.01,7.9,305.0,...,0.0,good,-3.01,2i06,A:305,"B:15,C:14",A_1,B_1.C_1,0.5,0.37


In [3]:
set_uni_prot = set()
for i in range(len(df)):
    up_id = df.loc[i]['UniProt']
    set_uni_prot.add(up_id)
print(len(set_uni_prot))

781


In [4]:
n = len(df)
ss = set()
for i in range(n):
    rdna_seq = df.loc[i]['nucleotide_sequence']
    for it in rdna_seq:
        ss.add(it)
ss

{'A', 'C', 'G', 'T', 'U', '|'}

In [5]:
n = len(df)
ss = set()
n_double_seq = 0
n_single_seq = 0
for i in range(n):
    rdna_seq = df.loc[i]['nucleotide_sequence']
    if '|' in rdna_seq:
        n_double_seq+=1
    else:
        n_single_seq+=1
n_double_seq, n_single_seq

(7095, 10166)

In [6]:
def binarize_categorical(matrix, n_classes, out=None):
    L = matrix.shape[0]
    matrix = matrix.astype(np.int32)
    if out is None:
        out = np.zeros([L, n_classes], dtype=np.bool_)
    subset = (matrix>=0) & (matrix<n_classes)
    out[np.arange(L)[subset],matrix[subset]] = 1
    return out

In [7]:
rdna_seq = df.loc[0]['nucleotide_sequence']


rdna_map = {'A':0, 'T':1, 'C':2, 'G':3, 'U':3}
rdna_seqs = None
if '|' in rdna_seq:
    rdna_seqs = rdna_seq.split('|')
else:
    rdna_seqs = [rdna_seq]

rdna_attributes = list()
for seq in rdna_seqs:
    seq_int = [rdna_map[x] for x in seq]
    seq_int = np.array(seq_int)
    seq_mat = binarize_categorical(seq_int, 4).astype(np.float32)
    print(seq_mat.shape)
    rdna_attributes.append(seq_mat)
np.concatenate(rdna_attributes).shape

(9, 4)
(9, 4)


(18, 4)

In [8]:
# data_file = 'seq_dg_identity_230404.csv'
data_file = 'table_hox_zscore_102107.txt'
data_file = os.path.join(data_dir, data_file)
df = pd.read_csv(data_file, sep='\t')
df.head(3)

,8mer,8mer.1,Alx3_3418.2,Alx4_1744.1,Arx_1738.2,Bapx1_2343.1,Barhl1_2590.2,Barhl2_3868.1,Barx1_2877.1,Barx2_3447.2,...,Tcf1_2666.2,Tcf2_0913.2,Tgif1_2342.2,Tgif2_3451.1,Titf1_1722.2,Tlx2_3498.2,Uncx4.1_2281.2,Vax1_3499.1,Vax2_3500.1,Vsx1_1728.1
0,AAAAAAAA,TTTTTTTT,2.3401,1.7044,2.5305,0.7962,1.8823,3.2222,1.2282,2.0546,...,1.4632,-0.0636,1.2235,0.0742,2.0043,0.5475,1.3708,1.6994,2.2566,3.3771
1,AAAAAAAC,GTTTTTTT,1.3416,1.4467,0.9565,0.0567,1.8291,2.4403,0.9911,1.2143,...,1.1287,1.0444,-0.2575,0.9416,0.3597,-0.2338,0.9116,1.2902,0.9958,1.2923
2,AAAAAAAG,CTTTTTTT,1.3386,1.0929,1.4749,1.1892,1.0593,1.4076,0.3329,1.2362,...,0.5170,0.4963,0.3452,0.2785,2.1461,-0.3770,0.3261,1.0687,0.3439,2.3979


In [9]:
prots = [x for x in df.keys()][2:]
prots = [x.split('_')[0] for x in prots]
print(prots)

['Alx3', 'Alx4', 'Arx', 'Bapx1', 'Barhl1', 'Barhl2', 'Barx1', 'Barx2', 'Bsx', 'Cart1', 'Cart1', 'Cdx1', 'Cdx2', 'Cphx', 'Crx', 'Cutl1', 'Cutl1', 'Dbx1', 'Dbx2', 'Dlx1', 'Dlx2', 'Dlx3', 'Dlx4', 'Dlx5', 'Dmbx1', 'Dobox4', 'Dobox5', 'Duxl', 'Emx2', 'En1', 'En2', 'Esx1', 'Evx1', 'Evx2', 'Gbx1', 'Gbx2', 'Gsc', 'Gsh2', 'Hdx', 'Hlx1', 'Hlxb9', 'Hmbox1', 'Hmx1', 'Hmx2', 'Hmx3', 'Homez', 'Hoxa1', 'Hoxa10', 'Hoxa11', 'Hoxa13', 'Hoxa2', 'Hoxa3', 'Hoxa4', 'Hoxa5', 'Hoxa6', 'Hoxa7', 'Hoxa7', 'Hoxa9', 'Hoxb13', 'Hoxb3', 'Hoxb4', 'Hoxb5', 'Hoxb6', 'Hoxb7', 'Hoxb8', 'Hoxb9', 'Hoxc10', 'Hoxc11', 'Hoxc12', 'Hoxc13', 'Hoxc4', 'Hoxc5', 'Hoxc6', 'Hoxc8', 'Hoxc9', 'Hoxd1', 'Hoxd10', 'Hoxd11', 'Hoxd12', 'Hoxd13', 'Hoxd3', 'Hoxd8', 'Ipf1', 'Irx2', 'Irx3', 'Irx3', 'Irx4', 'Irx5', 'Irx6', 'Isl2', 'Isx', 'Lbx2', 'Lhx1', 'Lhx2', 'Lhx3', 'Lhx4', 'Lhx5', 'Lhx6', 'Lhx6', 'Lhx8', 'Lhx9', 'Lmx1a', 'Lmx1b', 'Meis1', 'Meox1', 'Mrg1', 'Mrg2', 'Msx1', 'Msx2', 'Msx3', 'Nkx1-1', 'Nkx1-2', 'Nkx2-2', 'Nkx2-3', 'Nkx2-4', 'Nkx2

In [19]:
data_dir2 = '_datasets'
data_file = 'homeodomains_mouse.fa'
data_file = os.path.join(data_dir, data_dir2, data_file)
f = open(data_file, 'r')
Lines = f.readlines()

prot2seq = dict()
name = None
for line in Lines:
    if line[0] == '>':
        name = line[1:-1]
    else:
        if name is not None and name in prots:
            prot2seq[name] = line[:-1]
        name = None
map2seq = {x:prot2seq[x] for x in prots if x in prot2seq.keys()}

In [23]:
prot_keys = [x for x in df.keys()][2:]
mapping = {
    'protein_sequence' : [],
    'nucleotide_sequence' : [],
    'zscore' : [],
}
for i in range(len(df)):
    
    # items = [df.loc[i][k] for k in prot_keys if k.split('_')[0] in map2seq.keys()]
    for k in prot_keys:
        orig_k = k
        k = k.split('_')[0]
        if k in map2seq.keys():
            p_seq = map2seq[k]
            n_seq = df.loc[i]['8mer'] + '|' + df.loc[i]['8mer.1']
            z = df.loc[i][orig_k]
            mapping['protein_sequence'].append(p_seq)
            mapping['nucleotide_sequence'].append(n_seq)
            mapping['zscore'].append(z)
            
out_df = pd.DataFrame(mapping)
out_df.to_csv('../data/hox_zscore.csv', index=False)